In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-134727")
exp = Experiment(workspace=ws, name="optimizeml-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RZX747X8Y to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-134727
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-134727


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

ws = Workspace.from_config() # this automatically looks for a directory .azureml

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig
# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(1,10,100),
    "--max_iter": choice(50,100,150)
})
# Specify a Policy
bpolicy =  BanditPolicy(slack_factor = 0.1, evaluation_interval=1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", entry_script="train.py", compute_target=cpu_cluster_name, use_docker=False)

environment = Environment.get(workspace=ws, name="AzureML-AutoML")
script_run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    environment=environment,
    compute_target=cpu_cluster_name
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=bpolicy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=80,
                             max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hd_config, show_output=True)
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run.register_model(model_name="bankmarketing_model",model_path="outputs/bestrunmodel.joblib")
#best_run.download_file(best_run.get_file_names()[-1], output_file_path="./outputs/")

#best_run.register_model(model_path='./outputs/model.joblib', model_name='bankmarketing_model', tags={"Training Context":"Estimator", "Script Context":"Parameters"},
                 # properties={"accuracy":best_run.get_metrics()["accuracy"]})

Model(workspace=Workspace.create(name='quick-starts-ws-134727', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-134727'), name=bankmarketing_model, id=bankmarketing_model:1, version=1, tags={}, properties={})

In [7]:
#best_run = hyperdrive_run.get_best_run_by_primary_metric()
#joblib.dump(value=best_run,filename="bestrun.joblib")
best_run.get_metrics()

{'Regularization Strength:': 100.0,
 'Max iterations:': 50,
 'accuracy': 0.9072837632776934}

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv",validate=True,
        include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False
    )

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split


# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="AUC_weighted",
    compute_target=cpu_cluster,
    enable_onnx_compatible_models=True,
    training_data=ds,
    label_column_name="y",
    n_cross_validations=5)

In [12]:
# Submit your automl run

experiment = Experiment(ws, "automl_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)



Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_24c10d02-42ef-40c9-95d9-ea669a6075b7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Generating individually featurized CV splits.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias

In [13]:
# Retrieve and save your best automl model.

automl_bestrun, automl_bestmodel = automl_run.get_output()


automl_bestrun.register_model(model_name="automl_bestmodel.pkl",model_path="./outputs/")


Model(workspace=Workspace.create(name='quick-starts-ws-134727', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-134727'), name=automl_bestmodel.pkl, id=automl_bestmodel.pkl:1, version=1, tags={}, properties={})

In [23]:
#automl_bestrun.get_tags()
automl_bestrun.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl_experiment\',\'compute_target\':\'cpu-cluster\',\'subscription_id\':\'976ee174-3882-4721-b90a-b5fef6b72f24\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_24c10d02-42ef-40c9-95d9-ea669a6075b7_24","experiment_name":"automl_experiment","workspace_name":"quick-starts-ws-134727","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group_name":"aml-quickstarts-134727"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '24',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.Compu

In [30]:
automl_bestrun.get_metrics()

{'average_precision_score_weighted': 0.9567603493151342,
 'log_loss': 0.17160993479203826,
 'recall_score_macro': 0.7470776626951169,
 'matthews_correlation': 0.5407740687046443,
 'recall_score_micro': 0.9152959028831564,
 'recall_score_weighted': 0.9152959028831564,
 'AUC_micro': 0.9813537916694489,
 'average_precision_score_macro': 0.8293929611704323,
 'weighted_accuracy': 0.9570768834869648,
 'f1_score_macro': 0.7683374100042548,
 'f1_score_micro': 0.9152959028831564,
 'precision_score_micro': 0.9152959028831564,
 'precision_score_weighted': 0.909352865761616,
 'AUC_macro': 0.9497447408619287,
 'balanced_accuracy': 0.7470776626951169,
 'norm_macro_recall': 0.4941553253902338,
 'f1_score_weighted': 0.9115096834098064,
 'accuracy': 0.9152959028831564,
 'precision_score_macro': 0.7959733523919279,
 'AUC_weighted': 0.9497447408619287,
 'average_precision_score_micro': 0.9821560772110193,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_24c10d02-42ef-40c9-95d9-ea669a6075b

In [31]:
cpu_cluster.delete()
cpu_cluster.wait_for_completion(show_output=True)

DeletingCurrent provisioning state of AmlCompute is "Deleting"



ComputeTargetException: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name cpu-cluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name cpu-cluster not found in provided workspace"
    }
}